# ಸರಳ ಚಿತ್ರ ವರ್ಗೀಕರಣೆ

ಈ ನೋಟ್ಬುಕ್ ನಿಮಗೆ ಪೂರ್ವ-ತರಬೇತಿಗೊಂಡ ನ್ಯೂರಲ್ ನೆಟ್‌ವರ್ಕ್ ಬಳಸಿ ಚಿತ್ರಗಳನ್ನು ವರ್ಗೀಕರಿಸುವ ವಿಧಾನವನ್ನು ತೋರಿಸುತ್ತದೆ.

**ನೀವು ಕಲಿಯುವದು:**
- ಪೂರ್ವ-ತರಬೇತಿಗೊಂಡ ಮಾದರಿಯನ್ನು ಹೇಗೆ ಲೋಡ್ ಮಾಡಿ ಬಳಸುವುದು
- ಚಿತ್ರ ಪೂರ್ವಸಿದ್ಧತೆ
- ಚಿತ್ರಗಳ ಮೇಲೆ ಭವಿಷ್ಯವಾಣಿ ಮಾಡುವುದು
- ವಿಶ್ವಾಸ ಅಂಕಗಳನ್ನು ಅರ್ಥಮಾಡಿಕೊಳ್ಳುವುದು

**ಬಳಕೆ ಪ್ರಕರಣ:** ಚಿತ್ರಗಳಲ್ಲಿ ವಸ್ತುಗಳನ್ನು ಗುರುತಿಸುವುದು (ಉದಾಹರಣೆಗೆ "ಬೆಕ್ಕು", "ನಾಯಿ", "ಕಾರ್" ಇತ್ಯಾದಿ)

---


## ಹಂತ 1: ಅಗತ್ಯ ಗ್ರಂಥಾಲಯಗಳನ್ನು ಆಮದುಮಾಡಿ

ನಾವು ಬೇಕಾದ ಸಾಧನಗಳನ್ನು ಆಮದುಮಾಡೋಣ. ಇವುಗಳೆಲ್ಲವನ್ನು ಇನ್ನೂ ಅರ್ಥಮಾಡಿಕೊಳ್ಳದಿದ್ದರೂ ಚಿಂತೆ ಬೇಡ!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## ಹಂತ 2: ಪೂರ್ವ-ಪ್ರಶಿಕ್ಷಿತ ಮಾದರಿಯನ್ನು ಲೋಡ್ ಮಾಡುವುದು

ನಾವು **MobileNetV2** ಅನ್ನು ಬಳಸಲಿದ್ದೇವೆ, ಇದು ಲಕ್ಷಾಂತರ ಚಿತ್ರಗಳ ಮೇಲೆ ಈಗಾಗಲೇ ತರಬೇತಿ ಪಡೆದಿರುವ ನ್ಯೂರಲ್ ನೆಟ್‌ವರ್ಕ್.

ಇದನ್ನು **ಟ್ರಾನ್ಸ್‌ಫರ್ ಲರ್ನಿಂಗ್** ಎಂದು ಕರೆಯುತ್ತಾರೆ - ಯಾರೋ ಇನ್ನೊಬ್ಬರು ತರಬೇತಿ ನೀಡಿದ ಮಾದರಿಯನ್ನು ಬಳಸುವುದು!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## ಹಂತ 3: ಸಹಾಯಕ ಕಾರ್ಯಗಳು

ನಮ್ಮ ಮಾದರಿಗಾಗಿ ಚಿತ್ರಗಳನ್ನು ಲೋಡ್ ಮಾಡಿ ಸಿದ್ಧಪಡಿಸುವ ಕಾರ್ಯಗಳನ್ನು ರಚಿಸೋಣ.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## ಹಂತ 4: ಮಾದರಿ ಚಿತ್ರಗಳ ಮೇಲೆ ಪರೀಕ್ಷೆ ಮಾಡಿ

ಇಂಟರ್ನೆಟ್‌ನಿಂದ ಕೆಲವು ಚಿತ್ರಗಳನ್ನು ವರ್ಗೀಕರಿಸಲು ಪ್ರಯತ್ನಿಸೋಣ!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### ಪ್ರತಿ ಚಿತ್ರವನ್ನು ವರ್ಗೀಕರಿಸಿ


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## ಹಂತ 5: ನಿಮ್ಮದೇ ಚಿತ್ರಗಳನ್ನು ಪ್ರಯತ್ನಿಸಿ!

ವರ್ಗೀಕರಿಸಲು ನೀವು ಬಯಸುವ ಯಾವುದೇ ಚಿತ್ರ URL ಅನ್ನು ಕೆಳಗಿನ URL ನೊಂದಿಗೆ ಬದಲಾಯಿಸಿ.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 ಏನು刚刚 ಸಂಭವಿಸಿತು?

1. **ನಾವು ಪೂರ್ವ-ಪ್ರಶಿಕ್ಷಿತ ಮಾದರಿಯನ್ನು ಲೋಡ್ ಮಾಡಿದೆವು** - MobileNetV2 ಲಕ್ಷಾಂತರ ಚಿತ್ರಗಳ ಮೇಲೆ ತರಬೇತಿಗೊಂಡಿದೆ  
2. **ನಾವು ಚಿತ್ರಗಳನ್ನು ಪೂರ್ವಸಿದ್ಧಗೊಳಿಸಿದ್ದೇವೆ** - ಮಾದರಿಗಾಗಿ ಅವುಗಳನ್ನು ಮರುಗಾತ್ರಗೊಳಿಸಿ ಸ್ವರೂಪಗೊಳಿಸಿದ್ದೇವೆ  
3. **ಮಾದರಿ ಭವಿಷ್ಯವಾಣಿ ಮಾಡಿತು** - 1000 ವಸ್ತು ವರ್ಗಗಳ ಸಾಧ್ಯತೆಗಳನ್ನು ನೀಡಿತು  
4. **ನಾವು ಫಲಿತಾಂಶಗಳನ್ನು ಡಿಕೋಡ್ ಮಾಡಿದೆವು** - ಸಂಖ್ಯೆಗಳನ್ನಿಂದ ಮಾನವ ಓದಲು ಸುಲಭವಾದ ಲೇಬಲ್ಗಳಿಗೆ ಪರಿವರ್ತನೆ ಮಾಡಿದೆವು  

### ವಿಶ್ವಾಸ ಅಂಕಗಳನ್ನು ಅರ್ಥಮಾಡಿಕೊಳ್ಳುವುದು

- **90-100%**: ತುಂಬಾ ವಿಶ್ವಾಸವಿದೆ (ಬಹುಶಃ ಖಚಿತವಾಗಿ ಸರಿಯಾಗಿದೆ)  
- **70-90%**: ವಿಶ್ವಾಸವಿದೆ (ಬಹುಶಃ ಸರಿಯಾಗಿದೆ)  
- **50-70%**: ಸ್ವಲ್ಪ ವಿಶ್ವಾಸವಿದೆ (ಸರಿಯಾಗಿರಬಹುದು)  
- **50% ಕ್ಕಿಂತ ಕಡಿಮೆ**: ಹೆಚ್ಚು ವಿಶ್ವಾಸವಿಲ್ಲ (ಅನಿಶ್ಚಿತ)  

### ಭವಿಷ್ಯವಾಣಿಗಳು ತಪ್ಪಾಗುವ ಕಾರಣಗಳು ಏನು?

- **ಅಸಾಮಾನ್ಯ ಕೋನ ಅಥವಾ ಬೆಳಕು** - ಮಾದರಿ ಸಾಮಾನ್ಯ ಫೋಟೋಗಳ ಮೇಲೆ ತರಬೇತಿಗೊಂಡಿದೆ  
- **ಬಹು ವಸ್ತುಗಳು** - ಮಾದರಿ ಒಂದು ಮುಖ್ಯ ವಸ್ತುವನ್ನು ನಿರೀಕ್ಷಿಸುತ್ತದೆ  
- **ಅಸಾಮಾನ್ಯ ವಸ್ತುಗಳು** - ಮಾದರಿ ಕೇವಲ 1000 ವರ್ಗಗಳನ್ನು ಮಾತ್ರ ತಿಳಿದುಕೊಳ್ಳುತ್ತದೆ  
- **ಕಡಿಮೆ ಗುಣಮಟ್ಟದ ಚಿತ್ರ** - ಮಸುಕಾದ ಅಥವಾ ಪಿಕ್ಸೆಲೇಟೆಡ್ ಚಿತ್ರಗಳು ಕಷ್ಟಕರವಾಗಿವೆ  

---


## 🚀 ಮುಂದಿನ ಹಂತಗಳು

1. **ವಿಭಿನ್ನ ಚಿತ್ರಗಳನ್ನು ಪ್ರಯತ್ನಿಸಿ:**
   - [Unsplash](https://unsplash.com) ನಲ್ಲಿ ಚಿತ್ರಗಳನ್ನು ಹುಡುಕಿ
   - ಬಲ ಕ್ಲಿಕ್ ಮಾಡಿ → "ಚಿತ್ರ ವಿಳಾಸ ನಕಲಿಸಿ" ಮೂಲಕ URL ಪಡೆಯಿರಿ

2. **ಪ್ರಯೋಗ ಮಾಡಿ:**
   - ಅಬ್ಸ್ಟ್ರಾಕ್ಟ್ ಆರ್ಟ್‌ನೊಂದಿಗೆ ಏನು ಆಗುತ್ತದೆ?
   - ಇದು ವಿಭಿನ್ನ ಕೋನಗಳಿಂದ ವಸ್ತುಗಳನ್ನು ಗುರುತಿಸಬಹುದೇ?
   - ಇದು ಬಹು ವಸ್ತುಗಳನ್ನು ಹೇಗೆ ನಿರ್ವಹಿಸುತ್ತದೆ?

3. **ಹೆಚ್ಚು ತಿಳಿದುಕೊಳ್ಳಿ:**
   - [ಕಂಪ್ಯೂಟರ್ ವೀಷನ್ ಪಾಠಗಳು](../lessons/4-ComputerVision/README.md) ಅನ್ವೇಷಿಸಿ
   - ನಿಮ್ಮದೇ ಚಿತ್ರ ವರ್ಗೀಕರಣಕಾರಿಯನ್ನು ತರಬೇತುಗೊಳಿಸುವುದನ್ನು ಕಲಿಯಿರಿ
   - CNNs (ಕನ್ವಲ್ಯೂಷನಲ್ ನ್ಯೂರಲ್ ನೆಟ್‌ವರ್ಕ್‌ಗಳು) ಹೇಗೆ ಕೆಲಸ ಮಾಡುತ್ತವೆ ಎಂಬುದನ್ನು ಅರ್ಥಮಾಡಿಕೊಳ್ಳಿ

---

## 🎉 ಅಭಿನಂದನೆಗಳು!

ನೀವು ಇತ್ತೀಚಿನ ತಂತ್ರಜ್ಞಾನ ನ್ಯೂರಲ್ ನೆಟ್‌ವರ್ಕ್ ಬಳಸಿ ಚಿತ್ರ ವರ್ಗೀಕರಣಕಾರಿಯನ್ನು ನಿರ್ಮಿಸಿದ್ದೀರಿ!

ಈ ತಂತ್ರಜ್ಞಾನವು ಈ ಕೆಳಗಿನ ಕ್ಷೇತ್ರಗಳಲ್ಲಿ ಬಳಸಲಾಗುತ್ತದೆ:
- Google Photos (ನಿಮ್ಮ ಫೋಟೋಗಳನ್ನು ವ್ಯವಸ್ಥಿತಗೊಳಿಸುವುದು)
- ಸ್ವಯಂಚಾಲಿತ ಕಾರುಗಳು (ವಸ್ತುಗಳನ್ನು ಗುರುತಿಸುವುದು)
- ವೈದ್ಯಕೀಯ ನಿರ್ಣಯ (X-ರೇಗಳನ್ನು ವಿಶ್ಲೇಷಿಸುವುದು)
- ಗುಣಮಟ್ಟ ನಿಯಂತ್ರಣ (ದೋಷಗಳನ್ನು ಪತ್ತೆಹಚ್ಚುವುದು)

ಮುಂದುವರಿದು ಅನ್ವೇಷಿಸಿ ಮತ್ತು ಕಲಿಯಿರಿ! 🚀


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ಅಸ್ವೀಕರಣ**:  
ಈ ದಸ್ತಾವೇಜು [Co-op Translator](https://github.com/Azure/co-op-translator) ಎಂಬ AI ಅನುವಾದ ಸೇವೆಯನ್ನು ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ಶುದ್ಧತೆಯತ್ತ ಪ್ರಯತ್ನಿಸುತ್ತಿದ್ದರೂ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ತಪ್ಪುಗಳು ಅಥವಾ ಅಸತ್ಯತೆಗಳು ಇರಬಹುದು ಎಂದು ದಯವಿಟ್ಟು ಗಮನಿಸಿ. ಮೂಲ ಭಾಷೆಯಲ್ಲಿರುವ ಮೂಲ ದಸ್ತಾವೇಜನ್ನು ಅಧಿಕೃತ ಮೂಲವೆಂದು ಪರಿಗಣಿಸಬೇಕು. ಮಹತ್ವದ ಮಾಹಿತಿಗಾಗಿ, ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದ ಬಳಕೆಯಿಂದ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪು ಅರ್ಥಮಾಡಿಕೊಳ್ಳುವಿಕೆ ಅಥವಾ ತಪ್ಪು ವಿವರಣೆಗಳಿಗೆ ನಾವು ಹೊಣೆಗಾರರಾಗುವುದಿಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
